In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import json
import os
import scipy.signal

from spykshrk.realtime.simulator import nspike_data

from spykshrk.franklab.pp_decoder.util import gaussian, normal2D, apply_no_anim_boundary, simplify_pos_pandas
from spykshrk.franklab.pp_decoder.pp_clusterless import OfflinePPDecoder
from spykshrk.franklab.pp_decoder.data_containers import EncodeSettings, DecodeSettings, SpikeObservation, \
                                                         LinearPosition, StimLockout

from spykshrk.franklab.pp_decoder.visualization import DecodeVisualizer

        
%load_ext Cython

%matplotlib inline

#pd.set_option('float_format', '{:,.2f}'.format)
pd.set_option('display.precision', 4)
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 15)
#pd.set_option('display.width', 80)

idx = pd.IndexSlice


In [2]:
%%time
# Load merged rec HDF store based on config

#config_file = '/opt/data36/daliu/realtime/spykshrk/ripple_dec/bond.config.json'
#config_file = '/opt/data36/daliu/realtime/spykshrk/dec_60uv/bond.config.json'
config_file = '/home/daliu/Src/spykshrk_realtime/config/bond_single.json'
config = json.load(open(config_file, 'r'))

# Main hdf5 data source file name
hdf_file = os.path.join(config['files']['output_dir'],
                        '{}.rec_merged.h5'.format(config['files']['prefix']))

# Extract just encode and decode settings from config
encode_settings = EncodeSettings(config)
decode_settings = DecodeSettings(config)

# Open data file
store = pd.HDFStore(hdf_file, mode='r')

# Encapsulate Spike Observation panda table in container
observ_obj = SpikeObservation.create_default(store['rec_3'])

# Grab stimulation lockout times
stim_lockout = StimLockout.create_default(store['rec_11'])

# Grab animal linearized real position
nspike_anim = nspike_data.AnimalInfo(**config['simulator']['nspike_animal_info'])
pos = nspike_data.PosMatDataStream(nspike_anim)
pos_data = pos.data

# Encapsulate linear position
lin_obj = LinearPosition.from_nspike_posmat(pos_data, encode_settings)

In [3]:
%%time
# Run PP decoding algorithm

time_bin_size = 300

decoder = OfflinePPDecoder(lin_obj=lin_obj, observ_obj=observ_obj,
                           encode_settings=encode_settings, decode_settings=decode_settings, 
                           which_trans_mat='learned')

posteriors = decoder.run_decoder()

In [4]:
posteriors

In [6]:
## Plot posteriors

plt_ranges = [[2461 + 250, 2461 + 400]]
             
for plt_range in plt_ranges:
    
    plt.figure(figsize=[200,10])
    ax = DecodeVisualizer.plot_decode_image(posteriors, plt_range, encode_settings)
    DecodeVisualizer.plot_linear_pos(lin_obj, plt_range)
    DecodeVisualizer.plot_stim_lockout(stim_lockout, plt_range, encode_settings.arm_coordinates[2][1] + 10)
    
    plt.xlim(plt_range)
plt.show()

In [ ]:
pd.pivot_table(decoder.observ_obj.data, index=['dec_bin', 'dec_bin_start', 'timestamp', 'ntrode_id'])

In [ ]:
%%time

dec_est_map = x_bins[np.argmax(posteriors, axis=1)]
dec_est_pos = pd.DataFrame({'est_pos': dec_est_map}, index=pd.Index(data=dec_bin_timesd, name='timestamp'))

pos_data_bins = bin_pos_data(pos_data_linpos, dec_bin_size)

In [ ]:
center_dec_error, left_dec_error, right_dec_error = calc_error_table(pos_data_bins, dec_est_pos,
                                                                     arm_coordinates, 0)

In [ ]:
plot_arms_error(center_dec_error, left_dec_error, right_dec_error, [2350, 3400])
plt.show()

In [ ]:
print('median error center: {:0.5}, left: {:0.5}, right: {:.5}'.format(np.median(center_dec_error['abs_error']),
                                                                       np.median(left_dec_error['abs_error']),
                                                                       np.median(right_dec_error['abs_error'])))

print('mean error center: {:0.5}, left: {:0.5}, right: {:.5}'.format(np.mean(center_dec_error['abs_error']),
                                                                     np.mean(left_dec_error['abs_error']),
                                                                     np.mean(right_dec_error['abs_error'])))